# Lab 4 - Extending Logistic Regression

### Eric Smith and Jake Carlson

## Introduction
For this lab, we will again be examining the Global Terrorism Database maintained by the National Consortium for the Study of Terrorism and Responses to Terrorism (START) at the University of Maryland. We will be looking at attacks that happened in the United States over the whole time span of the data set, since it's creation in 1974.

We will be extending binary logistic regression based classification to support multiple classes. Specifically, based on the attributes of the input entity, we want to predict what the attack type of the entity is.

## Business Understanding

### Motivations
Protecting the United States from terror threats has been a major objective of the federal government. This is characterized by the founding of the Department of Homeland Security in 2001. But predicting when an attack will happen based on certain attributes is next to impossible. Attempting to train a model on the Global Terrorism Database to learn when terrorist attacks happen will result in a model that is over-trained on the GTD and will fail to predict any such attacks. Not to mention, such a system would have to be accompanied by a large-scale communication monitoring and processing system capable of feeding the model relevant inputs that exemplify a possible attack.

Instead of trying to predict when an attack will happen, our goal is to create a model that can predict the most likely attack type given a set of attributes about a possible attack. Such a model would allow city officials and law enforcement to define a set of attack circumstances that they are concerned about and predict the attack method that would be used to carry out that attack. Knowing the most likely method of attack would allow city officials and law enforcement to plan for such attacks. Law enforcement could practice how to mitigate the aftermath of an attack and secure an area. Allowing law enforcement to train on the most probable types of attacks for a location would maximize the utilization of training time, generating the most return for the community in terms of tax dollars allocated to training exercises and confidence in the local law enforcement's ability to manage an attack scenario.

As discovered in Lab 1, each city has a unique signature of the frequency of each attack it sees, as well as the types of targets that are most commonly attacked. Developing a model like the one described here would help enable local authorities to more appropriately tailor their training programs to their cities.

Common tactical response training programs focus on training students for a broad range of attack types. These type of programs are still important, as law enforcement should be prepared for any scenario that could potentialy affect their city, but if local police engage in multiple rounds of training through out the year, subsequent training sessions can focus on the types of attacks they are most likely to see.

### Objectives
Based on the characteristics of an attack, such as the target type and the date, we want to assign an attack type label to the entity. Because our system will be used to augment the training procedures for local law enforcement, perfect classification of attack scenarios is not required. Our desired accuracy is ...

...

## Data Preparation
Here is the list of attributes we will keep in our data set to use for classification.

#### General Information
- **year** (ordinal): The year the event occured in
- **month** (ordinal): The month the event occured in
- **day** (ordinal): The day the event occured in
- **extended** (nominal/binary): 1 if the incident was longer than 24 hours, 0 otherwise
    - **resolution** (ordinal): The date an extended incident was resolved if *extended* is 1


- **inclusion criteria** (nominal/binary): There are three inclusion criteria where a 1 indicates the event meets that criteria
    - **crit1**: Political, economic, religious, or social goal
    - **crit2**: Intention to coerce, intimidate, or publicize
    - **crit3**: Outside international humanitarian law


#### Location
We will provide the name of the city to the model. An alternative method would be to train a unique logistic regression algorithm for each city where our system is deployed.
- **city** (text): Name of the city in which the event occured
- **vicinity** (nominal/binary): A 1 indicates the event occured in the immediate vicinity of *city*, 0 indicates the even occured in *city*

#### Attack Type
The most severe method of attack. This will be our class label. Although the original data set contains columns for three different attack types, the attack types are ranked by their severity. Many attacks only have one attack type. By removing the second and third attack types from our data set, we will still be predicting the most severe of the attack types.
- **attacktype1, attacktype1_txt** (ordinal): Most severe attack type

- The attack types follow the following hierarchy:
    1. Assassination
    2. Armed Assault
    3. Bombing/Explosion
    4. Hijacking
    5. Barricade Incident
    6. Kidnapping
    7. Facility/Infrastructure Attack 
    8. Unarmed Assault
    9. Unknown


- **success** (nominal/binary): Indicates whether or the attack was successful (1) or unsuccessful (0)

#### Target Type
We will only be considering the first target type of the attack. The set of target attributes is provided below:
- **targtype1, targtype1_txt** (nominal): The general type of target from the following list:
    1. Business
    2. Government (General)
    3. Police
    4. Military
    5. Abortion related
    6. Airports and aircraft
    7. Government (Diplomatic)
    8. Educational institution
    9. Food or water supply
    10. Journalists and media
    11. Maritime
    12. NGO
    13. Other
    14. Private citizens and property
    15. Religious figures and institutions
    16. Telecommunication
    17. Terrirists and non-state militias
    18. Tourists
    19. Transportation
    20. Unknown
    21. Utilities
    22. Violent political parties
    

- ? **targsubtype1, targsubtype1_txt** (nominal): There are a number of subtypes for each of the above target types

#### Perpetrator Information
The data set provides information on up to three perpetrators if the attack was conducted by multiple groups. We will only be considering the first group, or the one decided to have the most responsibility for the attack.
- **gname** (text): The name of the group that carried out the attack
- **individual** (nominal/binary): A 1 indicates the individuals carrying out the attack are not affiliated with a terror organization
- **nperps** (oridnal): Indicates the total number of terrorists participating in the event
- **nperpcap** (oridnal): Number of perpatrators taken into custody

#### Casualties and Consequences
- **nkill** (ordinal): Records the number of confirmed kills for the incident
- **nkillter** (ordinal): Indicates the number of terrorists who were killed in the event
- **nwound** (ordinal): Indicates the number of people who sustained non-fatal injuries in the event
- **nwoundte** (ordinal): Indicates the number of terrorists who sustained non-lethal injuries
- **property** (nominal/binary): A 1 indicates the event resulted in property damage
- **propextent, propextent_txt** (nominal): If *property* is a 1, this field records the extent of the property damage following the scheme:
    1. Catastrophic (likely > \$1 billion)
    2. Major (likely > \$1 million and < \$1 billion)
    3. Minor (likely < \$1 million)
    4. Unknown


- **propvalue** (ordinal): Total value in USD of the property damages
